In [2]:
import requests
import json

api_server_url = "http://localhost:21122"
# api_server_url = "https://allsecure.co.kr/llm"
# api_server_url = "http://localhost:41001"

# 1. Controller API

### GET /list_models 

In [5]:
ret = requests.get(api_server_url + "/list_models")

output_json = ret.json()
print(output_json)

{'models': []}


### GET /refresh_all_models

In [4]:
ret = requests.get(api_server_url + "/refresh_all_models")

### GET /get_random_uuid

In [132]:
ret = requests.get(api_server_url + "/get_random_uuid")

output_json = ret.json()
print(output_json)

{'uuid': '48a5a28aaf554537bcf9eaa8ebc64739'}


### (deprecated) {controller_address}/get_worker_address

model_name = output_json['models'][1]
print("model_name:", model_name)

input_json = {
    "model": model_name
}

ret = requests.post(
    controller_url + "/get_worker_address", 
    json=input_json
)

output_json = ret.json()
print(output_json)

# 2. Worker API

### POST /model_details 

In [133]:
input_json = {
    "model_name": "OLLM-Small_2024.01.16",
}

ret = requests.post(
    api_server_url + "/model_details",
    json=input_json,
)

output_json = ret.json()
print(output_json)

{'context_length': 4096}


### POST /worker_get_conv_template  

In [124]:
input_json = {
    "model_name": "OLLM-Small_2024.01.16",
}

ret = requests.post(
    api_server_url + "/worker_get_conv_template",
    json=input_json,
)

output_json = ret.json()
print(output_json)

{'conv': {'name': 'chat-orca', 'system_template': '{system_message}', 'system_message': '### System:\nYou are an AI assistant, please behave and help the user.', 'roles': ['### User', '### Assistant'], 'messages': [], 'offset': 0, 'sep_style': 2, 'sep': '\n\n', 'sep2': '</s>', 'stop_str': ['</s>'], 'stop_token_ids': None, 'tasks': {'retrieval': '### System:\nYou are an AI assistant, please behave and help the user.\nDate:{date}\nYour reply should be based on the context below.\n\nContext:{instruction}', 'instruct': '### System:\nYou are an AI assistant, please behave and help the user. Your reply should be based on the context below.\n\nContext:{instruction}', 'system_instruct': '### System:\n{system}', 'correction': '', 'summarization': '### System:\nYou are an AI assistant, Summarize below sentences.', 'enkotranslation': '### System:\nYou are an AI translator, who knows every language and how to translate one language to another. convert english sentences to korean sentences. do not 

### POST /count_token 

In [453]:
input_json = {
    "model_name": "PIE-72B-20000-GPTQ", #"OLLM-Small_2024.01.16",
    # "prompt": "### System:\nThis is a system prompt, please behave and help the user.\n\n### User: NFC는 무엇의 약자입니까?\n\n### Assistant:"
    "prompt": prompt,
}

ret = requests.post(api_server_url + "/count_token", json=input_json)

output_json = ret.json()
print(output_json)

{'count': 1683, 'error_code': 0}


### POST /worker_generate

In [419]:
dummy = """Q: What command do I use to find the size of all the files (recursively) in a Linux or Mac OS X directory?\nA: The BSD version of ```du``` used in OS X reports size with 512-byte blocks -- the sizes are essentially rounded up to the next 512-byte value. This tells you the space on disk, which is larger than the amount of data. If you have a lot of small files, the difference can be large.\nHere's an example.\nThis is the value with regular ```du```. It's in 512-byte blocks:\n```$ du -s\n248   .\n```\nThe ```-h``` flag results in a more readable number, in kilobytes. As expected, it's half the number of 512-byte blocks:\n```$ du -hs\n124K  .\n```\nFinally, you can use ```find``` and ```awk``` to give you the sum of actual bytes in the files. This is kind of slow, but it works:\n```$ find . -type f -exec ls -l {} \\; | awk '{sum += $5} END {print sum}'\n60527\n```\nThis value matches exactly the number reported by Finder's Get Info window. (There are no weird forks or xattrs in this set of files.) It's significantly smaller than the value reported by ```du```.\nHere's how it works: it gets a list of all the files, and passes them to ```ls -l```; then ```awk``` is used to count up the bytes. The ```-type f``` flag is there so that only files (and not directories) get sent to ```ls```. Without that flag, it'll also send directory names to ```ls```, and each file will be listed twice : once as an individual file, and once as an item in the directory.\nThe GNU version of ```du``` can give values in actual bytes instead of blocks. It's unfortunate that the BSD version of ```du``` is not as flexible."""
prompt = f"### 영어:\n{dummy}\n### 한국어:\n"
input_json = {
    "model_name": "OLLM-Small_2024.01.16",
    "prompt": prompt,
    "temperature": 0.7,
    "top_p": 0.8,
    # "session_id": 3,
    "max_new_tokens": 4096,
    "stop": ["</끝>", "###"],
}

ret = requests.post(
    api_server_url + "/worker_generate",
    json=input_json,
)

output_json = ret.json()
# print(output_json)
output_json['text'][len(prompt):].rstrip('\n')

"Q: Linux 또는 Mac OS X 디렉터리의 모든 파일 크기를 (재귀적으로) 찾는 명령어는 무엇입니까?\nA: OS X에서 사용되는 BSD 버전의 `du`는 512바이트 블록으로 크기를 보고합니다. 즉, 크기는 실제 데이터보다 디스크 공간을 나타내며, 512바이트 단위로 반올림됩니다. 작은 파일이 많으면 차이가 크게 될 수 있습니다.\n다음은 예제입니다.\n일반 `du`를 사용한 값은 다음과 같습니다. 512바이트 블록으로 표시됩니다:\n```$ du -s\n248   .\n```\n`-h` 플래그는 더 읽기 쉬운 형식인 킬로바이트로 숫자를 표시합니다. 예상대로 512바이트 블록 수의 절반입니다:\n```$ du -hs\n124K  .\n```\n마지막으로 `find`와 `awk`를 사용하여 파일의 실제 바이트 합계를 구할 수 있습니다. 이 방법은 조금 느리지만 작동합니다:\n```$ find . -type f -exec ls -l {} \\; | awk '{sum += $5} END {print sum}'\n60527\n```\n이 값은 Finder의 Get Info 창에 보고된 숫자와 정확히 일치합니다. (이 파일 세트에는 이상한 포크 또는 xattrs가 없습니다.) `du`에 의해 보고된 값보다 상당히 작습니다.\n작동 방식은 다음과 같습니다: 모든 파일 목록을 얻은 후 `ls -l`에 전달하고, `awk`를 사용하여 바이트를 계산합니다. `-type f` 플래그는 파일(디렉터리가 아님)만 `ls`에 전달되도록 하는 데 사용됩니다. 이 플래그가 없으면 디렉터리 이름도 `ls`에 전달되고, 각 파일은 개별 파일로 한 번, 디렉터리의 항목으로 다시 한 번 나열됩니다.\nGNU 버전의 `du`는 블록 대신 실제 바이트로 값을 제공할 수 있습니다. 아쉽게도 BSD 버전의 `du`는 그렇게 유연하지 않습니다."

### POST /worker_generate_stream

In [473]:
# input_json = {
#     "model_name": "MingAI-70B-chat-orca_v0.42_2_dpo-GPTQ",
#     "prompt": "### System:\nThis is a system prompt, please behave and help the user.\n\n### User: react hello world\n\n### Assistant: 반응 프로젝트에서 \"Hello World\"를 표시하려면 다음 단계를 따르십시오.\n1. 새로운 React 응용 프로그램 생성 :\n먼저 새로운 React 응용 프로그램을 만들어야합니다.\n 다음 명령을 실행하여 새로운 React 앱을 만듭니다.\n```\nNPX Create-React-App My-App\n```\n이 명령은 새로운 폴더`My-App '에서 새로운 React 프로젝트를 만듭니다.\n2. 구성 요소 만들기 :\n이제 앱의 루트 구성 요소를 만들어야합니다.\n 터미널에서 다음 명령을 실행하여 새 구성 요소를 만듭니다.\n```\nCD My-App\nNPX React-S 생성 구성 요소 환경 적 응용 프로그램\n```\n이 명령은`src` 디렉토리에`myapp.js`라는 새 파���일을 생성합니다.\n3. 구성 요소 업데이트 :\n`myapp.js` 파일을 열고 다음 코드를 추가하십시오.\n```JS\n'React'에서 React React React;\n함수 myapp () {\n   반환 (\n     <h1> 안녕하세요, 세계! </h1>\n   );\n}\n수출 기본 MyApp;\n```\n이 코드는 \"Hello, World!\"텍스트를 표시하는 간단한 구성 요소를 만듭니다.\n4. 구성 요소 렌더링 :\n이제 앱의 기본 구성 요소를 렌더링해야합니다.\n `src/index.js` 파일을 열고 코드를 업데이트하십시오.\n```JS\n'React'에서 React React React;\n'./myapp'에서 myapp 가져 오기;\n기능 실행 () {\n   반환 (\n     <MyApp />\n   );\n}\n기본 기능 내보내기;\n```\n5. 앱 실행 :\n마지막으로 앱을 실행해보세요.\n```\nNPX 시작\n```\n이렇게하면 웹 브라우저에서 응용 프로그램을 실행하는 데 사용할 수있는 개발 서버가 시작됩니다.\n 응용 프로그램을 열고 브라우저에서 \"Hello, World!\"텍스트를 표시해야합니다.\n번역 번역\n반응 반응 프로젝트 프로젝트에서에서 \"hello world\"를 표시하려면하려면 다음",
#     "max_new_tokens": 1024,
#     "temperature": 0.7,
#     "top_p": 0.8,
#     "session_id": 3,
#     "repetition_penalty": 1.7,
# }
user_input = """

"""

instruct = (
    "You are a New draft generator with merged documents. "
    "User will give you Title and Multiple documents. "
    "You must reply only with a merged draft with reference to the documents provided. "
    "do not write explanations."
)
docus = [
    """1. 실험 제목 
 공기 저항력에 대한 고찰은 낙하 운동을 하는 물체에서 발생하는 현상을 다루는 학문적 분석이다. (12 words) 

2. 실험 목적 
 지구의 질량에 의한 중력과 대기의 저항 때문에 떨어지는 물체들의 속도, 가속도, 질량, 저항력 간의 관계를 확인하기 위한 실험을 설명하고 있다. 동일 기준 단면적을 가진 서로 다른 질량의 물체들이 떨어질 때의 위치 정보를 측정하여 속도, 가속도를 계산하고 공기 저항력을 계산하여 상관관계를 확인한다. 

3. 이론 및 원리 
 등가속도 운동은 일정한 가속도로 움직이는 운동으로, 시간 속도 그래프에서 일정한 기울기를 가진 직선 형태를 보입니다. 이 운동에서 적용되는 공식은 등가속도 운동 공식들로, 자유낙하 운동은 이 운동의 대표적인 예시입니다. 실험에서 떨어지는 물체의 위치 정보를 측정하는 초음파센서는 초음파를 사용하여 센서와 물체 사이의 거리를 측정합니다. 초음파센서의 거리 측정 공식은 수신된 초음파의 시간을 이용하여 거리를 계산합니다. 

4. 장치 및 방법 
 필요한 물품: 초음파 센서, 가드, 사각판, 850인터페이스, PC (초음파 센서는 위치 정보, 가드는 보호, 850인터페이스는 데이터 전송, PC는 데이터 처리와 분석 및 그래프 생성)
- 초음파 센서를 테이블에 위치시키고 수평을 조절(물체와 수평 맞추기)
- 센서에 보호용 가드 설치
- 센서 연결선을 인터페이스 입력에 연결
- capstone 프로그램 실행(샘플링률 100Hz 사용)

5. 실험 결과 
 실험 1의 실험 결과로 추세선 식의 이계도함수를 이용해 평균 가속도를 구하고, 이를 공기 저항력과 관련된 식에 대입하여 공기 저항력을 계산했습니다. 이 과정을 통해 실험 1에서 공기 저항력은 약 0.123836 kg·m/s^2 이 됩니다. 

실험 2의 실험 결과로 추세선 식의 이계도함수를 이용해 평균 가속도를 구하고, 이를 공기 저항력과 관련된 식에 대입하여 공기 저항력을 계산했습니다. 이 과정을 통해 실험 2에서 공기 저항력은 약 0.1413058 kg·m/s^2 이 됩니다. 

두 실험에서 각각 계산한 공기 저항력의 차이는 약 14.1% 수준이라는 것을 알 수 있습니다. 

6. 결론 및 토의 
 실험 결과로 더 많은 질량을 가진 사각판은 더 큰 공기 저항력을 보였고, 이는 가속도 운동에서 속도가 증가함에 따라 공기 저항력도 증가하며 결국 중력의 힘과 공기 저항력이 같아지는 지점이 생긴다는 가설이 나온다. 그러나 실험에서 관찰된 일차항과 상수항은 사람의 손으로 실험을 진행한 결과로 발생한 오차로 추정된다. 초음파 센서는 발신된 펄스 덩어리가 난반사되어 수신기로 도달하지 못할 수 있으며, 센서와 물체의 수평이 맞지 않을 경우 오차가 발생할 수 있다. 실험에서 capstone 프로그램은 초음파 센서의 위치 값을 이용해 가속도를 계산하지만, 초음파 센서의 정보는 완벽한 이차 곡선을 보이지 않아 미분 과정에서 오차가 증폭될 수 있다. 

7. 참고 문헌 
 다avid Halliday, Robert Resnick, Jearl Walker의 "Principles of Physics" 11판, 제목에 따라 24~29쪽과 11쪽을 요약하자면 없습니다. 이 문장만으로는 요약할 내용이 없습니다.
    """,
    """1. 실험제목 
 가속도 측정 실험 중 하나인 등가속도운동을 요약하면, 물체가 직선적인 가속도로 일정한 속도로 움직이는 운동을 의미한다. 이 운동은 가속도를 측정하는데 사용되며, 실험적으로 가속도를 측정하여 이 운동의 특성을 확인할 수 있다. 

2. 실험목적 
 영상분석을 이용해 에어트랙에서 추에 매달린 물체의 운동을 분석하여 뉴턴의 운동 법칙을 확인한다. 

3. 실험결과 
 첫 번째 실험에서 추의 질량은 14.81g, 글라이더의 질량은 192.17g이며, 추의 질량은 2회 실험에서 14.81g, 글라이더의 질량은 212.15g이고 셋째 실험에서 추의 질량은 25.29g, 글라이더의 질량은 212.15g이다. 추와 글라이더의 가속도는 각각 64.73, 58.392, 97.494 cm/s²이며, 계산된 가속도는 각각 70.12, 63.94, 104.38 cm/s²이고 상대 오차는 각각 7.686%, 8.676%, 6.597%이다. 추와 글라이더의 운동은 제곱에 비례하는 추세로 나타난다. 

4. 실험결과의 분석 
 직선 운동하는 물체의 가속도를 뉴턴 법칙과 실험 결과를 이용해 측정하고 비교했다. 실험 결과, 추가된 외부 요인으로 인해 가속도 측정치는 약간 작게 나타났지만, 상대 오차는 작아 뉴턴 법칙이 잘 적용됐다는 것을 확인했다. 
    """,
]

title = "등가속도운동과 그 예시 실험들"

document_prompt = ""
for didx, docu in enumerate(docus):
    document_prompt += f"Document {didx+1}:\n```\n{docu}\n```\n\n"

prompt = f"""### System:
{instruct}
### User: I want you to act as Document Merger. please create a draft with refering my Documents and Title. The draft must be a good mix of Documents.
{document_prompt}
Title: {title}
Language: 한국어

### Assistant:"""

prompt = f"""
<|im_start|>system
{instruct}<|im_end|>
<|im_start|>user
I want you to act as New draft generator with merged documents. I will give you Document 1, Document 2. You must reply with a new draft titled "{title}" with refering below Document 1, Document 2. You should merge Document 1 and Document 2 into a sigle draft.
{document_prompt}
Language: 한국어<|im_end|>
<|im_start|>assistant
"""

In [472]:
input_json = {
    "model_name":  "PIE-72B-20000-GPTQ", #"M-DIE-M-10B_gpt4_ep3-yarn-64k", #"OLLM-Small_2024.01.16", #"MDIEM-toc3",#"OLLM-Large_2023.11.20", # , #"DIE_10.7b_v0.1_ep2","DIE-MoE-10.7Bx4_v8_gpt_ep3"
    "prompt": prompt,
    "temperature": 0.8,
    "top_p": 0.7,
    # "session_id": 3,
    "max_new_tokens": 4096,
    # "stop": ["</끝>", "###"],
    "stop": ["<|endoftext|>", "<|im_start|>", "<|im_end|>"],
}

ret = requests.post(
    api_server_url + "/worker_generate_stream",
    json=input_json,
    stream=True,
    # timeout=200,
)

past_len = len(input_json["prompt"])
for chunk in ret.iter_lines(decode_unicode=False, delimiter=b"\0"):
    if chunk:
        data = json.loads(chunk.decode())
        print(data["text"][past_len:], sep="", end="")
        past_len = len(data["text"])

등가속도운동과 그 예시 실험들

1. 실험 제목
공기 저항력에 대한 고찰은 낙하 운동을 하는 물체에서 발생하는 현상을 다루는 학문적 분석이다.
2. 실험 목적
지구의 질량에 의한 중력과 대기의 저항 때문에 떨어지는 물체들의 속도, 가속도, 질량, 저항력 간의 관계를 확인하기 위한 실험을 설명하고 있다. 동일 기준 단면적을 가진 서로 다른 질량의 물체들이 떨어질 때의 위치 정보를 측정하여 속도, 가속도를 계산하고 공기 저항력을 계산하여 상관관계를 확인한다.
3. 이론 및 원리
등가속도 운동은 일정한 가속도로 움직이는 운동으로, 시간 속도 그래프에서 일정한 기울기를 가진 직선 형태를 보입니다. 이 운동에서 적용되는 공식은 등가속도 운동 공식들로, 자유낙하 운동은 이 운동의 대표적인 예시입니다. 실험에서 떨어지는 물체의 위치 정보를 측정하는 초음파센서는 초음파를 사용하여 센서와 물체 사이의 거리를 측정합니다. 초음파센서의 거리 측정 공식은 수신된 초음파의 시간을 이용하여 거리를 계산합니다.
4. 장치 및 방법
필요한 물품: 초음파 센서, 가드, 사각판, 850인터페이스, PC (초음파 센서는 위치 정보, 가드는 보호, 850인터페이스는 데이터 전송, PC는 데이터 처리와 분석 및 그래프 생성)
- 초음파 센서를 테이블에 위치시키고 수평을 조절(물체와 수평 맞추기)
- 센서에 보호용 가드 설치
- 센서 연결선을 인터페이스 입력에 연결
- capstone 프로그램 실행(샘플링률 100Hz 사용)
5. 실험 결과
실험 1의 실험 결과로 추세선 식의 이계도함수를 이용해 평균 가속도를 구하고, 이를 공기 저항력과 관련된 식에 대입하여 공기 저항력을 계산했습니다. 이 과정을 통해 실험 1에서 공기 저항력은 약 0.123836 kg·m/s^2 이 됩니다.
실험 2의 실험 결과로 추세선 식의 이계도함수를 이용해 평균 가속도를 구하고, 이를 공기 저항력과 관련된 식에 대입하여 공기 저항력을 계산했습니다. 이 과정을 통해 실험 2에서 공기 저항력은 약 0.1413058 kg·m/s^

In [470]:
print(prompt)


<|im_start|>system
You are a New draft generator with merged documents. User will give you Title and Multiple documents. You must reply only with a merged draft with reference to the documents provided. do not write explanations.<|im_end|>
<|im_start|>user
I want you to act as New draft generator with merged documents. I will give you Document 1, Document 2. You must reply with a new draft titled "등가속도운동과 그 예시 실험들" with refering below Document 1, Document 2. You should merge Document 1 and Document 2 into a sigle document.
Document 1:
```
1. 실험 제목 
 공기 저항력에 대한 고찰은 낙하 운동을 하는 물체에서 발생하는 현상을 다루는 학문적 분석이다. (12 words) 

2. 실험 목적 
 지구의 질량에 의한 중력과 대기의 저항 때문에 떨어지는 물체들의 속도, 가속도, 질량, 저항력 간의 관계를 확인하기 위한 실험을 설명하고 있다. 동일 기준 단면적을 가진 서로 다른 질량의 물체들이 떨어질 때의 위치 정보를 측정하여 속도, 가속도를 계산하고 공기 저항력을 계산하여 상관관계를 확인한다. 

3. 이론 및 원리 
 등가속도 운동은 일정한 가속도로 움직이는 운동으로, 시간 속도 그래프에서 일정한 기울기를 가진 직선 형태를 보입니다. 이 운동에서 적용되는 공식은 등가속도 운동 공식들로, 자유낙하 운동은 이 운동의 대표적인 예시입니다. 실험에서 떨어지는 물체의 위치 정보를 측정하는 초음파센서는 초음파를 사용하여 센서와 물체 사이의 거리를 측정

### POST /worker_generate_auto

In [370]:
input_json = {
    "model_name": "OLLM-Small_2024.01.16",
    "user_input": """반가워요""",
    "session_id": 20,
    "temperature": 0.7,
    "top_p": 0.8,
    "max_new_tokens": 4096,
    "stop": ["</끝>", "###"],
}

ret = requests.post(
    api_server_url + "/worker_generate_auto",
    json=input_json,
)

output_json = ret.json()
output_json['text']

'  네, 반가워요! 오늘 어떤 도움이 필요하신가요?'

In [139]:
print(output_json['text'])

Q: Linux 또는 Mac OS X 디렉터리에 있는 모든 파일의 크기를 재귀적으로 찾으려면 어떤 명령을 사용해야 하나요?
A: OS X에서 사용되는 BSD 버전의 ```du```는 512바이트 블록 단위로 크기를 보고하며, 크기는 기본적으로 다음 512바이트 값으로 반올림됩니다. 따라서 디스크 공간은 데이터의 양보다 더 큰 양입니다. 작은 파일이 많은 경우 그 차이가 클 수 있습니다.
다음은 예시입니다.
이것은 일반적으로 ```du```를 사용하는 값입니다. 512바이트 블록 단위로 표시됩니다:
```$ du -s
248 .
```
du -h``` 플래그를 사용하면 킬로바이트 단위로 더 읽기 쉬운 숫자가 표시됩니다. 예상대로 512바이트 블록의 절반 정도입니다:
```$ du -hs
124K .
```
마지막으로, ```find``` 및 ```awk```를 사용하여 파일에 있는 실제 바이트의 합계를 구할 수 있습니다. 이 방법은 속도가 느리지만 작동합니다:
```$ find . -type f -exec ls -l {} \; | awk '{sum += $5} END {print sum}'
60527
```
이 값은 Finder의 정보 확인 창에 표시되는 수치와 정확히 일치합니다. (이 파일 세트에는 이상한 포크나 xattr이 없습니다.) 이 값은 ```du```가 보고하는 값보다 훨씬 작습니다.
이 방법은 모든 파일 목록을 가져와서 ```ls -l```로 전달한 다음 ```awk```를 사용하여 바이트 수를 계산하는 것입니다. type f``` 플래그는 파일(디렉터리 제외)만 ```ls```로 전송되도록 하기 위해 사용됩니다. 이 플래그가 없으면 디렉터리 이름도 ```ls```로 전송되며, 각 파일은 개별 파일로 한 번, 디렉터리의 항목으로 한 번 등 두 번 나열됩니다.
du```의 GNU 버전은 블록 대신 실제 바이트 단위로 값을 제공할 수 있습니다. 안타깝게도 BSD 버전의 ```du```는 유연성이 떨어집니다.



### POST /worker_generate_stream_auto

In [404]:
input_json = {
    "model_name": "OLLM-Small_2024.01.16",
    "user_input": """지금 내 질문이 몇번째 질문이야?""",
    "session_id": 24,
    # "context": '\nDate:2023-11-13 15:21\nYour reply should be based on the context below.\n\nContext:\n(주)아도스 2023년 기업정보 | 사원수, 회사소개, 근무환경 ...\n(주)아도스 기업소개 - 업력 : 10년차, 기업형태 : 중소기업, 업종 : 응용 소프트웨어 개발 및 공급업 | (주)아도스의 사원수, 연봉, 채용, 복리후생, 재무정보 등이 ... 재산, 가치를 구하는 것을 사명으로, 좋은 사람들이 모여 즐겁게 성장하고 있는 회사입니다.\n1. 현재는 IT보안이 주 사업 영역이며, AI와 가상화기술 기반의 정보보호 솔루션을 개발/운영하고 있습니다.\n2. 대기업 제조사의 네트워크 보안, 생산라인 보안, 정보 유출 방지 등 통합보안 서비스(모의 해킹, 진단, 관제, 이상징후 분석)를 담당하고 있습니다.\n"인류의 생명과 가치를 구한다"\n우리는 단단하게 성장하고 있어요\n-\n2021\n-\n09월\n가드랜드(정보보호솔루션) 출시\n-\n03월\n가두리(웹격리솔루션) 출시\n- 09월\n-\n2020\n\n-\n09월\n병역지정업체(산업체) 선정\n-\n05월\n벤처기업확인(기술보증기금,유효기간:2022.05.28)\n-\n04월\n수차에 걸쳐 자본금을 1200백만원으로 증자\n- 09월\n-\n2019\n-\n12월\n기업부설연구소 설립\n- 12월\n-\n2017\n-\n10월\n상호를 (주)아도스로 변경\n- 10월\n-\n2014\n-\n09월\n컴퓨터 프로그래밍 서비스업을 목적으로 대표이사 이승원에 의해 (주)피그플라이 설립\n- 09월\n우리는 직원의 복리후생도 중요해요\n-\n지원금/보험\n- 각종 경조사 지원\n-\n급여제도\n- 퇴직연금\n- 인센티브제\n- 스톡옵션\n- 퇴직금\n-\n업\n- 대표자명\n- 이승원\n- 사업내용\n-\n소프트웨어 개발,게임 개발\n- 주소\n-\n서울 송파구 송파대로 167, B동 1021호지도보기\n- SNS\n-\n- 보유기술\n-\n-\n특허권\n- 디스플레이 해킹 방지 시스템 및 그 방법(2020)\n-\n특허권\n- 웹 클라이언트를 이용한 망 분리 시스템 및 그 방법(2020)\n-\n특허권\n- 컴퓨터의 시스템자원 및 프로세스의 보호 및 격리장치와 그방법(2020)\n-\n특허권\n- PKI 기반의 사물인터넷 기기 인증방법 및 인증시스템(2020)\n- 특허권\n미래 혹은 가까에 있는 위협으로 부터, 우리의 생명,\n\n\n(주)아도니스 2023년 기업정보 | 사원수, 회사소개, 근무환경 ...\n(주)아도니스 기업소개 - 업력 : 35년차, 기업형태 : -, 업종 : 골프장 운영업 | (주)아도니스의 사원수, 연봉, 채용, 복리후생, 재무정보 등이 궁금하시다면, ...- 사업내용\n-\n골프장,호텔 운영/무역,담배,골프용품 도소매,통신판매/한식,중식 음식점\n- 주소\n-\n경기 포천시 신북면 포천로 2499지도보기\n기업비전\n주요 사업내용 : 포천 아도니스CC 운영\n주요 사업내용 : 포천 아도니스CC 운영\n주요 취급품목 : 골프장운영, 호텔운영\n우리는 단단하게 성장하고 있어요\n-\n2005\n-\n01월\n호텔 준공 영업개시\n- 01월\n-\n1999\n-\n06월\n상호변경 : (주)대우레져 -> (주)아도니스\n-\n04월\n골프장 개장\n- 06월\n-\n1995\n-\n04월\n골프장 착공허가 (경기도 포천군)\n- 04월\n-\n1\n990\n-\n10월\n경기도로부터 골프장사업계획을 승인\n- 10월\n-\n1989\n-\n02월\n회사 설립\n- 02월\n인원 규모를 파악해 보세요!\n전체 직원 수 56명\n- 전체인원\n- 입사자\n- 퇴사자\n',
    "max_new_tokens": 4096,
    "temperature": 0.7,
    "top_p": 0.8,
}

ret = requests.post(
    api_server_url + "/worker_generate_stream_auto",
    json=input_json,
    stream=True,
    # timeout=2,
)

past_len = 0
for chunk in ret.iter_lines(decode_unicode=False, delimiter=b"\0"):
    if chunk:
        data = json.loads(chunk.decode())
        print(data["text"][past_len:], sep="", end="")
        past_len = len(data["text"])

 이 질문은 당신의 두 번째 질문입니다. 첫 번째 질문에 대한 답변을 위해 저는 여러분의 질문 내용을 분석하고 관련 정보를 제공하는 데 집중하겠습니다. 다른 도움이 필요하신 것이 있나요?

In [395]:
data

{'err_code': 10002}

### POST /worker_get_last_prompt

In [ ]:
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
    "session_id": 0,
}

ret = requests.post(
    api_server_url + "/worker_get_last_prompt",
    json=input_json,
)

output_json = ret.json()
print(output_json)

{'prompt': '### System:\nThis is a system prompt, please behave and help the user.\nDate:2023-11-13 15:21\nYour reply should be based on the context below.\n\nContext:\n(주)아도스 2023년 기업정보 | 사원수, 회사소개, 근무환경 ...\n(주)아도스 기업소개 - 업력 : 10년차, 기업형태 : 중소기업, 업종 : 응용 소프트웨어 개발 및 공급업 | (주)아도스의 사원수, 연봉, 채용, 복리후생, 재무정보 등이 ... 재산, 가치를 구하는 것을 사명으로, 좋은 사람들이 모여 즐겁게 성장하고 있는 회사입니다.\n1. 현재는 IT보안이 주 사업 영역이며, AI와 가상화기술 기반의 정보보호 솔루션을 개발/운영하고 있습니다.\n2. 대기업 제조사의 네트워크 보안, 생산라인 보안, 정보 유출 방지 등 통합보안 서비스(모의 해킹, 진단, 관제, 이상징후 분석)를 담당하고 있습니다.\n"인류의 생명과 가치를 구한다"\n우리는 단단하게 성장하고 있어요\n-\n2021\n-\n09월\n가드랜드(정보보호솔루션) 출시\n-\n03월\n가두리(웹격리솔루션) 출시\n- 09월\n-\n2020\n\n-\n09월\n병역지정업체(산업체) 선정\n-\n05월\n벤처기업확인(기술보증기금,유효기간:2022.05.28)\n-\n04월\n수차에 걸쳐 자본금을 1200백만원으로 증자\n- 09월\n-\n2019\n-\n12월\n기업부설연구소 설립\n- 12월\n-\n2017\n-\n10월\n상호를 (주)아도스로 변경\n- 10월\n-\n2014\n-\n09월\n컴퓨터 프로그래밍 서비스업을 목적으로 대표이사 이승원에 의해 (주)피그플라이 설립\n- 09월\n우리는 직원의 복리후생도 중요해요\n-\n지원금/보험\n- 각종 경조사 지원\n-\n급여제도\n- 퇴직연금\n- 인센티브제\n- 스톡옵션\n- 퇴직금\n-\n업\n- 대표자명\n- 이승원\n- 사업내용\n-\n소프

### POST /session_history_clear

In [12]:
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
    "session_id": 0,
}

ret = requests.post(
    api_server_url + "/session_history_clear",
    json=input_json,
)

output_json = ret.json()
print(output_json)

{'err_code': 0}


### POST /worker_regenerate_auto

In [13]:
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
    "session_id": 0,
}

ret = requests.post(
    api_server_url + "/worker_regenerate_auto",
    json=input_json,
)

output_json = ret.json()
print(output_json)

{'err_code': 70003}


### POST /worker_regenerate_stream_auto

In [14]:
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
    "session_id": 0,
}

ret = requests.post(
    api_server_url + "/worker_regenerate_stream_auto",
    json=input_json,
    stream=True,
)

# past_len = len(input_json["prompt"])
past_len = 0
for chunk in ret.iter_lines(decode_unicode=False, delimiter=b"\0"):
    if chunk:
        data = json.loads(chunk.decode())
        if "text" not in data:
            continue
        print(data["text"][past_len:], sep="", end="")
        past_len = len(data["text"])

### POST /worker_stop_stream

In [15]:
import uuid
from multiprocessing import Process
import time


# background runner for worker_generate_stream
def background_run_stream(session_id):
    input_json = {
        "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
        "prompt": "### System:\nThis is a system prompt, please behave and help the user.\n\n### User: 유튜브에 대해서 설명해주세요.\n\n### Assistant:",
        "session_id": session_id,
        "temperature": 0.7,
        "top_p": 0.8,
    }

    ret = requests.post(
        api_server_url + "/worker_generate_stream",
        json=input_json,
        headers={"User-Agent": "FastChat Client"},
        stream=True,
        timeout=200,
    )

    past_len = len(input_json["prompt"])
    for idx, chunk in enumerate(ret.iter_lines(decode_unicode=False, delimiter=b"\0")):
        if chunk:
            data = json.loads(chunk.decode())
            print(data["text"][past_len:], end="")
            past_len = len(data["text"])


# generate random session_id
session_id = uuid.uuid4().hex
print("session_id:", session_id)

# run worker_generate_stream
p1 = Process(target=background_run_stream, args=[session_id])
p1.start()

# wait for 3 seconds
time.sleep(3)

# send worker_stop_stream
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
    "session_id": session_id,
}

ret = requests.post(
    api_server_url + "/worker_stop_stream",
    json=input_json,
)

print("\n\n Generation Stopped")

session_id: b5bdc37a9ed14ac1b3d05e7ff5566659
 유튜브는 세계에서 가장 큰 온라인 동영상 공유 및 소셜 미디어 플

 Generation Stopped
